In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [5]:
#load the csv file
file_path="game.csv"
data=pd.read_csv(file_path)
data.head()

,userId,game,rating
0,3,The Legend of Zelda: Ocarina of Time,4.0
1,6,Tony Hawk's Pro Skater 2,5.0
2,8,Grand Theft Auto IV,4.0
3,10,SoulCalibur,4.0
4,11,Grand Theft Auto IV,4.5


In [9]:
#Step 1: Create a user-item matrix (rows:users,colimns:games,values:rating
user_item_matrix=data.pivot_table(index="userId",columns="game",values="rating")
"""
pivot table : this function reshapes the dataset into matrix where:

       Each row represents a user (identified by userId).
       Each column represents a game (idenfied by game).
       The value in the matrix represent the ratings that 
       users gave to the games
    """

'\npivot table : this function reshapes the dataset into matrix where:\n\n       Each row represents a user (identified by userId).\n       Each column represents a game (idenfied by game).\n       The value in the matrix represent the ratings that \n       users gave to the games\n    '

In [11]:
#step 2: Fill NaN values with 0 (assuming no rating means the game has not)

user_item_matrix_filled=user_item_matrix.fillna(0)
"""
This line represents any missing values (Nans)
in the user-item matrix with 0
indicating that the user did not rate that particular game.
"""


'\nThis line represents any missing values (Nans)\nin the user-item matrix with 0\nindicating that the user did not rate that particular game.\n'

In [15]:
#step 3: Compute the coseine similarity between the users based on raw rating
user_similarity=cosine_similarity(user_item_matrix_filled)

#convert similarity matrix to DataFrame  for easy reference 
user_similarity_df=pd.DataFrame(user_similarity,index=user_item_matrix.index,columns=user_item_matrix.index)

In [45]:
#step 4: Function to get game recommendation fora specific user based on

def get_collab_recomm_for_user(user_id,num_recommendations=5):
    #get similarity scores for the input user with all other users
    similar_users=user_similarity_df[user_id].sort_values(ascending=False)

    #get the most similar users(excluding the user themselves)
    similar_users=similar_users.drop(user_id)

    #select the top N similar users to limit noise (e.g top 50 users)
    top_similar_users=similar_users.head(50)

    #this selects the top similar user to limit noise in recommendation
    #Get rating of this similar users ,weighted bby their similarity scores
    weighted_ratings=np.dot(top_similar_users.values,user_item_matrix_filled.loc[top_similar_users.index])

    #np.dot:this computs thye dot product between the similarity scores of the top similar users and thier corresponding ratings in the 
    #user -item matrix .
    #The result is an array of weighted ratings for each game.
    #Normalize by the sum of similarities

    sum_of_similarities=top_similar_users.sum()

    if sum_of_similarities>0:
        weighted_ratings/=sum_of_similarities
    #the weigthed ratings are normalized by didivding by the sum 

    #Recommend games that user hasn't rated yet 
    user_ratings=user_item_matrix_filled.loc[user_id]
    unrated_games=user_ratings[user_ratings==0]
    #identifies games that target users has not rated 

    #get the weighted scores for unrated games
    game_recommendations=pd.Series(weighted_ratings, index=user_item_matrix_filled.columns).loc[unrated_games.index]
    #this creates a pandas series from the weighted ratings and filters to include only the unrated games
    #finally it sorts the recommended games in descending order and returns the top specified number of recommmendations

    #return the top 'num_recommendations' game recommendations
    return game_recommendations.sort_values(ascending=False).head(num_recommendations)

In [51]:
#Example usage : get recommendations for a user with 10 3
recommeded_games=get_collab_recomm_for_user(user_id=22)


In [53]:
#print the recommended games
print("Recommended games for user 3 :")
print(recommeded_games)

Recommended games for user 3 :
game
Alan Wake: The Writer                 0.728709
Command & Conquer 3: Tiberium Wars    0.666940
World of Tanks                        0.666940
'Splosion Man                         0.000000
Retro City Rampage                    0.000000
dtype: float64
